<h1>Computer Vision</h1>

                                                                                                Assignment 0
                                                                                                Archit Kumar
                                                                                                2018201051

#  Converting video into images

Following code converts the video "movie.mp4" into its constitutent images and save them into ./images folder  

<a href= "https://drive.google.com/file/d/1gVjPJzTdlS6uVo-0bdJjiLwhll1n5KCE/view?usp=sharing"> link of sample video</a>

In [9]:
import cv2 as cv

video = cv.VideoCapture('movie.mp4') 
status , image = video.read() 
count =  0 
while status :
	cv.imwrite("images/%d.jpg"%count , image)
	status , image = video.read()
	count += 1

<h4>sample output</h4>
<p><img src = "images/0.jpg">Frame 0</p>
<p><img src = "images/50.jpg">Frame 50</p>

# Merging a set of images into single video 

Following code works as follows :<br>
1. iterate through each image in the given folder ./images and stores the name in 'files' list in sorted order<br>
2. open each image in 'files' list using 'cv.imread' function and store into array.<br>
3. use video.writer function to write images into video buffer .<br>
4. finally save video using release function.<br>
5. Frame rate is controlled by <b>"fps"</b>

<h4> sample images</h4>
<img src = "images/0.jpg"><img src = "images/76.jpg">

In [12]:
import cv2 as cv
import numpy as np
import os
from os.path import isfile ,  join
import locale

path_in  = './images/'
path_out = 'video.mp4'
fps = 23.0 # Frame Per second

array = []
files = [int(f.split('.')[0]) for f in os.listdir(path_in)]
files.sort()

for i in range(len(files)):
	filename = path_in + '%d.jpg'%files[i]
	img = cv.imread(filename)
	height , width , layers = img.shape
	size = (width,height)
	array.append(img)
  
out = cv.VideoWriter(path_out , cv.VideoWriter_fourcc(*'mp4v') , fps , size)
for i in range(len(array)):
	out.write(array[i])
out.release()


<h4>output </h4>
<a href = "https://drive.google.com/file/d/1gVjPJzTdlS6uVo-0bdJjiLwhll1n5KCE/view?usp=sharing">link  for output video</a>

# Capturing Images from Webcam

Following code accesss webcam and show the images into a new window. on pressing space image is captured and stored into ./cam folder . on pressing escape window is closed. 

In [8]:
import cv2 as cv
cam = cv.VideoCapture(0)
cv.namedWindow("test")
img_counter = 0 

while True : 
	ret , frame = cam.read()
	cv.imshow("test",frame)
	if not ret :
		break
	k = cv.waitKey(1) #waiting for key press
	if k%256 == 27:
		print "Escape hit , closing"
		break
	elif k%256 == 32 :
		print "image is captured and saved into ./cam folder"
		cv.imwrite("./cam/%d.png"%img_counter,  frame)
		img_counter = img_counter + 1
cam.release()
cv.destroyAllWindows()

image is captured and saved into ./cam folder
Escape hit , closing


##  Chroma Keying  <br>

Chroma keying, is a visual effects/post-production technique for compositing (layering) two images or video streams together based on color hues.

Working of different functions are as follow : 

<h4> colorclose function</h4>
this function simply calculate a mask which will be used in merging.<br>
Input : A 1*3 array which corresponds to pixel value into [y cb cr] color space.<br>
output : 0 or 255 based on the closesness of input pixel with keycolor.<br>

<h4>result function</h4>
this function returns the merged frame<br>
Input : two image frame which corresponds to foreground and background video<br>
output : merged frame<br>
working : based on mask ($ \alpha $) obtained from colorclose function , pixel value of output image is calculated as follow :<br>
$$ out(i,j) = (1 - \alpha)*foreground(i,j) + \alpha*background(i,j) $$
<br><i>rest code simply captures frames from input video and output video in loop and output frame from "result" function is simply added into buffer needed for final video. </i>

<h4> Input videos link</h4>
<a href = "https://drive.google.com/file/d/1M57JBcu-XkP8hdBTiH280h_tvqKD_vm2/view?usp=sharing">background video</a><br>
<a href = "https://drive.google.com/file/d/1i6LdceJjSuoJV9EYdXH_04guXURbwfsl/view?usp=sharing">foreground video</a>

In [ ]:
import cv2
import numpy as np
import math
from PIL import Image


key_color = (149,44,21)
tolerance = [50,130]
[y_key , cb_key ,cr_key] = key_color 


def colorclose(arr):
	y , cb_p , cr_p = arr
	temp = math.sqrt((cb_key-cb_p)**2 + (cr_key - cr_p)**2)
	if temp < 100  :
	else:
		z = 1.0
	return 255.0*z

def result(foreground , backgronud):
	forycbcr = cv2.cvtColor(foreground, cv2.COLOR_BGR2YCrCb)
	alpha = np.apply_along_axis(colorclose , 2, forycbcr)
	alpha = np.uint8(255-255*(alpha/255))
	alpha = cv2.cvtColor(alpha, cv2.COLOR_GRAY2BGR)

	alpha = alpha.astype(float)/255
	foreground = foreground.astype(float)
	backgronud = backgronud.astype(float)

	foreground = cv2.multiply(1.0 - alpha, foreground)
	backgronud = cv2.multiply(alpha , backgronud)

	outImage = cv2.add(foreground , backgronud)
	return  outImage


vidcap1 = cv2.VideoCapture('jets.mp4')
vidcap2 = cv2.VideoCapture('back.mp4')
frame_array = []
fps = 15.0


success1 , image1 = vidcap1.read() 
success2 , image2 = vidcap2.read() 
count = 0
while success1 and success2 and count < 250:
	image1 = cv2.resize(image1 , (800,600), interpolation = cv2.INTER_AREA)
	image2 = cv2.resize(image2 , (800,600), interpolation = cv2.INTER_AREA)	
	result_img = result(image1 , image2)
	count = count+1

	print "merged %d frames" %count
	frame_array.append(result_img)
	success1 , image1 = vidcap1.read()
	success2 , image2 = vidcap2.read()
    
out = cv2.VideoWriter("merged.mp4", cv2.VideoWriter_fourcc(*'mp4v') , fps , (800,600))
for i in range(len(frame_array)):
	out.write(frame_array[i].astype(np.uint8))
out.release()


<h4>output video</h4>
<a href = "https://drive.google.com/file/d/1pV7rgMM7qmFhi8fi1yDZDu7AlXHTUpC7/view?usp=sharing"> merged video</a>